In [181]:
#import basic modules
import numpy as np
import pandas as pd
img_size = 48   
folder_path = r"C:\Users\Yash Bhargava\anaconda3\Data"  # Dataset folder path......Stored in 'Data' folder

listdir method of os module helps to get a list of avaiable files or folders in a directory.
We have out train and test images in train and test folder.

In [182]:
import os
os.listdir(folder_path) 

['test', 'train']

In [183]:
%%time
from tqdm import tqdm    # tqdm for loading gui
import cv2               # import opencv
def get_data(s):
    # s:train|test
    labels = os.listdir(f'{folder_path}/{s}')     # Sample path:-  "C:\Users\Yash Bhargava\anaconda3\Data\train"
    x = []
    y = []
    for label in labels:
        image_it = os.listdir(f'{folder_path}/{s}/{label}') # Sample path:-"C:\Users\Yash Bhargava\anaconda3\Data\train\angry"
        for img_name in tqdm(image_it):
            img_path = (f'{folder_path}/{s}/{label}/{img_name}') # Sample path:-"C:\Users\Yash Bhargava\anaconda3\Data\train\angry\imgname"                  
            img = cv2.imread(img_path)
            x.append(np.expand_dims(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY),axis=-1)) # Adding extra axis to dimension array as required by algo.
            y.append(label)
    return np.array(x),np.array(y)
train_image,train_label = get_data('train')   # store training images and labels in separate variables
test_image,test_label = get_data('test')      # store testing images and labels in sparate variables

100%|███████████████████████████████████████████████████████████████████████████████| 831/831 [00:01<00:00, 535.95it/s]

Wall time: 1min 13s


In [184]:
train_image.shape,train_label.shape,test_image.shape, test_label.shape

((28709, 48, 48, 1), (28709,), (7177, 48, 48, 1), (7177,))

In [185]:
pd.Series(train_label).value_counts(),pd.Series(test_label).value_counts()

(happy       7215
 neutral     4965
 sad         4830
 fear        4097
 angry       3995
 surprise    3171
 disgust      436
 dtype: int64,
 happy       1774
 sad         1247
 neutral     1233
 fear        1024
 angry        958
 surprise     831
 disgust      110
 dtype: int64)

In [186]:
train_label = pd.Series(train_label)
test_label = pd.Series(test_label)

In [187]:
train_label.factorize()

(array([0, 0, 0, ..., 6, 6, 6], dtype=int64),
 Index(['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'], dtype='object'))

In [188]:
test_label

0          angry
1          angry
2          angry
3          angry
4          angry
          ...   
7172    surprise
7173    surprise
7174    surprise
7175    surprise
7176    surprise
Length: 7177, dtype: object

In [189]:

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
train_image = train_image.astype('float32')
test_image = test_image.astype('float32')

# normalizing the data to help with the training
train_image /= 255
test_image /= 255

In [190]:
n_classes = 7
# Label Encoding
print("Shape before one-hot encoding: ", train_label.shape)
train_label = np_utils.to_categorical(train_label.factorize()[0], n_classes)
test_label = np_utils.to_categorical(test_label.factorize()[0], n_classes)
print("Shape after one-hot encoding: ", train_label.shape)
print(type(train_label))

Shape before one-hot encoding:  (28709,)
Shape after one-hot encoding:  (28709, 7)
<class 'numpy.ndarray'>


In [194]:
%%time
model = Sequential()
# convolutional layer
model.add(Conv2D(8, kernel_size=(2,2), strides=(1,1), padding = 'same', activation='relu', input_shape = (48,48,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(16, kernel_size=(2,2), strides=(1,1), padding = 'same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(32, kernel_size=(2,2), strides=(1,1), padding = 'same', activation='relu'))
# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(Conv2D(64, kernel_size=(2,2), strides=(1,1), padding = 'same', activation='relu'))
# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(Conv2D(128, kernel_size=(2,2), strides=(1,1), padding = 'same', activation='relu'))
# model.add(MaxPool2D(pool_size=(2,2)))

# flatten output of conv
model.add(Flatten())
# hidden layer
model.add(Dense(5000, activation='relu'))
# output layer
model.add(Dense(7, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') # Model Architecture Ends

# training the model for 30 epochs
model.fit(train_image, train_label, batch_size=64, epochs=50, validation_data=(test_image, test_label))


Epoch 1/50
449/449 [==============================] - 171s 378ms/step - loss: 1.6319 - accuracy: 0.3620 - val_loss: 1.4874 - val_accuracy: 0.4315
Epoch 2/50
449/449 [==============================] - 161s 358ms/step - loss: 1.3994 - accuracy: 0.4611 - val_loss: 1.3530 - val_accuracy: 0.4772
Epoch 3/50
449/449 [==============================] - 278s 619ms/step - loss: 1.2461 - accuracy: 0.5264 - val_loss: 1.3069 - val_accuracy: 0.4988
Epoch 4/50
449/449 [==============================] - 164s 366ms/step - loss: 1.0825 - accuracy: 0.5930 - val_loss: 1.2910 - val_accuracy: 0.5151
Epoch 5/50
449/449 [==============================] - 172s 383ms/step - loss: 0.8664 - accuracy: 0.6862 - val_loss: 1.3148 - val_accuracy: 0.5286
Epoch 6/50
449/449 [==============================] - 160s 356ms/step - loss: 0.5905 - accuracy: 0.7926 - val_loss: 1.4174 - val_accuracy: 0.5371
Epoch 7/50
449/449 [==============================] - 162s 362ms/step - loss: 0.3268 - accuracy: 0.8965 - val_loss: 1.7400 -

In [195]:
model.summary()

Model: "sequential_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_201 (Conv2D)         (None, 48, 48, 8)         40        
                                                                 
 max_pooling2d_199 (MaxPooli  (None, 24, 24, 8)        0         
 ng2D)                                                           
                                                                 
 conv2d_202 (Conv2D)         (None, 24, 24, 16)        528       
                                                                 
 max_pooling2d_200 (MaxPooli  (None, 12, 12, 16)       0         
 ng2D)                                                           
                                                                 
 conv2d_203 (Conv2D)         (None, 12, 12, 32)        2080      
                                                                 
 flatten_32 (Flatten)        (None, 4608)            

In [196]:
score = model.evaluate(test_image, test_label)
print("Test Score: ", score[0])
print("Test accuracy: ", score[1])

225/225 [==============================] - 11s 47ms/step - loss: 3.3025 - accuracy: 0.5327
Test Score:  3.3024911880493164
Test accuracy:  0.5326738357543945
